In [ ]:
import torch
import numpy as np
from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm
import pandas as pd
from collections import Counter

# MODEL
class BASELINE_simpleLSTMmodel(nn.Module):
    def __init__(self, dataset):
        super(BASELINE_simpleLSTMmodel, self).__init__()
        self.n_lstm_layers = 2
        self.words_emb_size = 128
        self.lstm_hidden_size = 128
        vocabulary_length = len(dataset.words_unique)

        self.embedding = nn.Embedding(num_embeddings=vocabulary_length, embedding_dim=self.words_emb_size)
        self.lstm = nn.LSTM(input_size=self.lstm_hidden_size, hidden_size=self.lstm_hidden_size, num_layers=self.n_lstm_layers)
        self.fc = nn.Linear(self.lstm_hidden_size, vocabulary_length)

    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        # predict logits for next word, no softmax since CrossEntropy has it inside
        logits = self.fc(output)
        return logits, state

In [ ]:
# DATASET
class NOBATCHES_dataset(torch.utils.data.Dataset):
    def __init__(self, seq_len, datapath):
        self.seq_len = seq_len
        self.datapath = datapath
        train_df = pd.read_csv(self.datapath)
        text = train_df['content'].str.cat(sep=' ')
        self.all_words = text.split()
        # store all words as one big list, works bad for big dataset, will make a good __getitem__ from disk next iteration

        word_counts = Counter(self.all_words)
        self.words_unique = sorted(word_counts, key=word_counts.get, reverse=True)

        self.i2w = {index: word for index, word in enumerate(self.words_unique)}
        self.w2i = {v:k for k,v in self.i2w.items()}
        self.indices = [self.w2i[w] for w in self.all_words]

    def __len__(self):
        return len(self.indices) - self.seq_len

    def __getitem__(self, index):
        return (
            torch.tensor(self.indices[index:index + self.seq_len]),
            torch.tensor(self.indices[index + 1:index + self.seq_len + 1]),
        )

In [ ]:
df = pd.read_csv('all.csv')
df2 = df.iloc[:5000]
df2.to_csv('train.csv', index=False)
dataset = NOBATCHES_dataset(10, 'train.csv')

In [45]:
for x,y in dataset:
    print('x looks like that')
    for token in x:
        print(dataset.i2w[int(token)])

    print()
    print('y, shifted 1 word forward, since we want to predict next word for each word in x')
    for token in y:
        print(dataset.i2w[int(token)])
    break

x looks like that
Недорого
окультуриваем
прямо
из
Санкт-Петербурга!Прививаем
любовь
к
театру,
формируем
литературную

y, shifted 1 word forward, since we want to predict next word for each word in x
окультуриваем
прямо
из
Санкт-Петербурга!Прививаем
любовь
к
театру,
формируем
литературную
зависимость,подсаживаем


In [ ]:
# it can be clearly seen that data is splitted bad for tokens, so we will fix it in next iteration

In [ ]:
batch_size = 200
max_epochs = 10
seq_len = 10
datapath = 'train.csv'  # try to use small file first

dataset = NOBATCHES_dataset(seq_len=seq_len, datapath=datapath)
model = BASELINE_simpleLSTMmodel(dataset)

model.train()
model = model.cuda()
dataloader = DataLoader(dataset, batch_size=batch_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(max_epochs):

    # init states to zero
    state_h = torch.zeros(model.n_lstm_layers, seq_len, model.lstm_hidden_size)
    state_c = torch.zeros(model.n_lstm_layers, seq_len, model.lstm_hidden_size)

    for batch, (x, y) in tqdm(enumerate(dataloader), total=len(dataloader)):
        optimizer.zero_grad()

        y_pred, (state_h, state_c) = model(x.cuda(), (state_h.cuda(), state_c.cuda()))
        loss = criterion(y_pred.transpose(1, 2), y.cuda())
        state_h = state_h.detach()
        state_c = state_c.detach()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            print({'loss': loss.item()})


In [48]:
# TRY TO generate some anecdotes
model.eval()
model = model.cpu()


def gen_anec(condition_text, n_trials, next_words):
    for trial in range(n_trials):
        words = condition_text.split(' ')
        state_h, state_c = model.init_state(len(words))

        for i in range(0, next_words):
            x = torch.tensor([[dataset.w2i[w] for w in words[i:]]])
            y_pred, (state_h, state_c) = model(x, (state_h, state_c))

            last_word_logits = y_pred[0][-1]
            softmaxed = torch.nn.functional.softmax(last_word_logits, dim=0).detach()

            ## uncomment for greedy word selection
            # maxind = torch.argmax(softmaxed)
            # words.append(dataset.index_to_word[int(maxind.detach())])

            ## uncomment for probabilistic selection of next word
            word_index = np.random.choice(len(last_word_logits), p=p)
            words.append(dataset.i2w[word_index])

        print(' '.join(words))


gen_anec('Заходит как то', 5, 10)
gen_anec('А что это вы', 5, 10)
gen_anec('Водка', 5, 10)

Заходит как то России Забегает полицейского пули шкафу прямоготелефона интрига протираются, того Кстати,
Заходит как то пробками! свободу ним обязаностям пиромана, было найдёте завод нашем нему
Заходит как то тридцать этом том,  стрелять забиваешь Оксана поддержать, нашей такой
Заходит как то относятся лес, месте. Г. вот!Кардиолог 4 того думал... мире прямоготелефона
Заходит как то Хорошо, золотую несколько страшный ведёрком подарить вас миллиардов некоторых Метро!-
А что это вы магазин скаку чемпионами Дмитрия России олимпийцы того,  смысле кухне
А что это вы Чмаровка, последний отличие замене морякирасстреляли здоровье! имени надпись К. Михаилу
А что это вы лосей Вовочка Москве Так, скамью пузо одном этом работе. инциндента
А что это вы 2 отсутствием Хабаровска блюдо, него Москве, жизнь сбываться.- примером было,
А что это вы "пенис" оказался!!!! тобой бегемота  том надзирателем. по-пластунски контрабандным, точно
Водка Тверской 60 пятеро ГИБДД года ушами" России левом Скрестили футб